In [34]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable

# hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 600
num_epochs = 10
learning_rate = 0.01

train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Processing...
Done!


In [35]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # input:28, hidden:128, num_layers:2
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # (2, 600, 128)==(num_layer, batch_size, hidden_size)
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        # output (600, 28, 128)==(batch, seq_length, hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [36]:
lstm = RNN(input_size, hidden_size, num_layers, num_classes)
lstm.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

In [39]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # images (batch, seq_length, input_size)==(600, 28, 28)
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()  
        labels = Variable(labels).cuda()
        
        optimizer.zero_grad()
        outputs = lstm(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))

Epoch [1/10], Loss: 0.0118
Epoch [2/10], Loss: 0.0373
Epoch [3/10], Loss: 0.0213
Epoch [4/10], Loss: 0.0256
Epoch [5/10], Loss: 0.0134
Epoch [6/10], Loss: 0.0131
Epoch [7/10], Loss: 0.0241
Epoch [8/10], Loss: 0.0172
Epoch [9/10], Loss: 0.0276
Epoch [10/10], Loss: 0.0245


In [26]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    outputs = lstm(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 96 %
